In [26]:
import pandas
import numpy
import os, sys
import json, pickle
from tqdm import tqdm

In [2]:
dataset_filepaths = dict()

In [4]:
path = '/Volumes/samsung/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv/'
dataset_filepaths['biorxiv_medrxiv'] = [os.path.join(path, e) for e in os.listdir(path) if e.endswith('.json')]

In [6]:
path = '/Volumes/samsung/2020-03-13/comm_use_subset/comm_use_subset/'
dataset_filepaths['comm_use_subset'] = [os.path.join(path, e) for e in os.listdir(path) if e.endswith('.json')]

In [7]:
path = '/Volumes/samsung/2020-03-13/noncomm_use_subset/noncomm_use_subset/'
dataset_filepaths['noncomm_use_subset'] = [os.path.join(path, e) for e in os.listdir(path) if e.endswith('.json')]

In [9]:
path = '/Volumes/samsung/2020-03-13/pmc_custom_license/pmc_custom_license/'
dataset_filepaths['pmc_custom_license'] = [os.path.join(path, e) for e in os.listdir(path) if e.endswith('.json')]

#### Reading a sample paper

In [10]:
with open(numpy.random.choice(dataset_filepaths['pmc_custom_license']), 'r') as handle:
    paper_data = json.load(handle)

In [14]:
a[e['section'] for e in paper_data['body_text']]

['SARS Epidemiology Modeling',
 'SARS Epidemiology Modeling',
 'SARS Epidemiology Modeling',
 'SARS Epidemiology Modeling',
 'SARS Epidemiology Modeling',
 'SARS Epidemiology Modeling',
 'Guofa Zhou* and Guiyan Yan*',
 'Diagnostic Criteria during SARS Outbreak in Hong Kong',
 'Diagnostic Criteria during SARS Outbreak in Hong Kong',
 'Diagnostic Criteria during SARS Outbreak in Hong Kong',
 'Diagnostic Criteria during SARS Outbreak in Hong Kong',
 'Diagnostic Criteria during SARS Outbreak in Hong Kong']

In [16]:
e = paper_data['body_text'][0]

In [17]:
e.keys()

dict_keys(['text', 'cite_spans', 'ref_spans', 'section'])

In [18]:
e['text']

'To the Editor: To assess the effectiveness of intervention measures during the recent severe acute respiratory syndrome (SARS) pandemic, Zhou and Yan (1) used Richards model, a logistic-type model, to fit the cumulative number of SARS cases reported daily in Singapore, Hong Kong, and Beijing. The key to using mathemati-cal models for SARS epidemiology is understanding the models (2) . In the Richards model (1) , the function F(S) in the model was described as measuring "the effectiveness of intervention measures." The parameters in F(S), namely, the maximum cases load K and the exponent of deviation a, depict the actual progression of the epidemic as described by the reported data. In other words, the parameter estimates are used to quantify end results of the intervention measures implemented during the outbreaks. Simply put, the all-important question of "what if?" was not answered by their result. To gauge the effectiveness of intervention measures, one should consider a more compl

In [19]:
from nltk.tokenize import sent_tokenize

In [27]:
len(sent_tokenize(e['text']))

8

In [23]:
str(paper_data['metadata'])

"{'title': '', 'authors': []}"

In [24]:
paper_data['paper_id']

'03e352c35635f746ff112e5a766eefe10efb59b4'

In [21]:
paper_data.keys()

dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])

In [36]:
final_dataset_dict = {
    'paper_id': [],
    'text_segment': [],
    'corresponding_section': [],
    'dataset': []
}

In [37]:
ignored_paper_count = 0
for dataset in dataset_filepaths.keys():
    print("processing dataset: {} \n".format(dataset))
    for filepath in tqdm(dataset_filepaths[dataset]):
        with open(filepath, 'r') as handle:
            try:
                paper_data = json.load(handle)
            except Exception as e:
                ignored_paper_count += 1
                continue
            paper_id = paper_data['paper_id']
            for e in paper_data['body_text']:
                section = e['section']
                for text_segment in sent_tokenize(e['text']):
                    final_dataset_dict['paper_id'].append(paper_id)
                    final_dataset_dict['text_segment'].append(text_segment)
                    final_dataset_dict['corresponding_section'].append(section)
                    final_dataset_dict['dataset'].append(dataset)

  0%|          | 8/1606 [00:00<00:25, 62.64it/s]

processing dataset: biorxiv_medrxiv 



  0%|          | 9/18000 [00:00<03:42, 80.97it/s]

processing dataset: comm_use_subset 



  0%|          | 1/3946 [00:00<08:02,  8.18it/s]

processing dataset: noncomm_use_subset 



  1%|          | 16/2852 [00:00<00:18, 153.69it/s]

processing dataset: pmc_custom_license 



100%|██████████| 2852/2852 [00:11<00:00, 240.68it/s]


In [40]:
final_dataset = pandas.DataFrame(final_dataset_dict)

In [ ]:
final_dataset.iloc[900:1000]

In [47]:
final_dataset.to_csv('/Volumes/samsung/2020-03-13/text_segment_dataset.csv')